In [1]:
#Importing libraries
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [2]:
#Reading data
df = pd.read_csv('D:/Mari/Data science material/Additional - Projects/DL/Churn model/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:

#Data summary
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
#Rownumber, customerID and Surname are having more unique values, so we can ignore this
df.drop(['RowNumber','CustomerId','Surname'], axis=1, inplace=True)
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
#Geography and Gender variable encoding
enc=pd.get_dummies(df[['Geography','Gender']], drop_first=True)

In [6]:
#Removing the already encoded variable in the existing dataframe
df.drop(['Gender', 'Geography'], axis=1, inplace=True)

In [7]:
#Concat encoded variable into existing dataframe
df = pd.concat([df,enc], axis=1)

In [8]:
#Separating Dependent and independent feature
Y = df['Exited']
X = df.drop('Exited',axis=1)

In [9]:
#Splitting the dataset into train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [10]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [11]:
def build_model(data):
    model = keras.Sequential()
    for i in range(data.Int('num_layers', 2,15)):
        model.add(layers.Dense(units=data.Int('units_'+str(i),
                                              min_value=32,
                                              max_value=512,
                                              step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(
        data.Choice('learning_rate', [1e-2,1e-3,1e-4])),
        loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [21]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project2',
    project_name='Churn prediction')

In [22]:

print(tuner.search_space_summary())

None


In [23]:
%%time
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
235/235 [==============================] - ETA: 0s - loss: 0.6934 - accuracy: 0.28 - ETA: 0s - loss: 0.5861 - accuracy: 0.77 - ETA: 0s - loss: 0.5207 - accuracy: 0.79 - ETA: 0s - loss: 0.4924 - accuracy: 0.79 - ETA: 0s - loss: 0.4728 - accuracy: 0.79 - ETA: 0s - loss: 0.4620 - accuracy: 0.79 - ETA: 0s - loss: 0.4517 - accuracy: 0.79 - ETA: 0s - loss: 0.4422 - accuracy: 0.80 - ETA: 0s - loss: 0.4339 - accuracy: 0.80 - 1s 3ms/step - loss: 0.4314 - accuracy: 0.8108 - val_loss: 0.3735 - val_accuracy: 0.8396
Epoch 2/50
235/235 [==============================] - ETA: 0s - loss: 0.5532 - accuracy: 0.75 - ETA: 0s - loss: 0.3612 - accuracy: 0.84 - ETA: 0s - loss: 0.3598 - accuracy: 0.84 - ETA: 0s - loss: 0.3589 - accuracy: 0.85 - ETA: 0s - loss: 0.3600 - accuracy: 0.85 - ETA: 0s - loss: 0.3631 - accuracy: 0.85 - ETA: 0s - loss: 0.3614 - accuracy: 0.85 - ETA: 0s - loss: 0.3608 - accuracy: 0.85 - ETA: 0s - loss: 0.3631 - accuracy: 0.85 - 1s 2ms/step - loss: 0.3616 - accuracy: 0.8519 - 

235/235 [==============================] - ETA: 0s - loss: 0.2744 - accuracy: 0.90 - ETA: 0s - loss: 0.2555 - accuracy: 0.90 - ETA: 0s - loss: 0.2690 - accuracy: 0.88 - ETA: 0s - loss: 0.2627 - accuracy: 0.89 - ETA: 0s - loss: 0.2678 - accuracy: 0.88 - ETA: 0s - loss: 0.2772 - accuracy: 0.88 - ETA: 0s - loss: 0.2769 - accuracy: 0.88 - ETA: 0s - loss: 0.2788 - accuracy: 0.88 - 0s 2ms/step - loss: 0.2817 - accuracy: 0.8827 - val_loss: 0.3603 - val_accuracy: 0.8536
Epoch 18/50
235/235 [==============================] - ETA: 0s - loss: 0.3093 - accuracy: 0.87 - ETA: 0s - loss: 0.2701 - accuracy: 0.89 - ETA: 0s - loss: 0.2674 - accuracy: 0.89 - ETA: 0s - loss: 0.2692 - accuracy: 0.88 - ETA: 0s - loss: 0.2690 - accuracy: 0.88 - ETA: 0s - loss: 0.2751 - accuracy: 0.88 - ETA: 0s - loss: 0.2703 - accuracy: 0.88 - ETA: 0s - loss: 0.2723 - accuracy: 0.88 - ETA: 0s - loss: 0.2761 - accuracy: 0.88 - 1s 2ms/step - loss: 0.2761 - accuracy: 0.8829 - val_loss: 0.3676 - val_accuracy: 0.8540
Epoch 19/50


235/235 [==============================] - ETA: 0s - loss: 0.1774 - accuracy: 0.90 - ETA: 0s - loss: 0.1547 - accuracy: 0.92 - ETA: 0s - loss: 0.1625 - accuracy: 0.92 - ETA: 0s - loss: 0.1663 - accuracy: 0.92 - ETA: 0s - loss: 0.1719 - accuracy: 0.91 - ETA: 0s - loss: 0.1743 - accuracy: 0.91 - ETA: 0s - loss: 0.1769 - accuracy: 0.91 - ETA: 0s - loss: 0.1816 - accuracy: 0.91 - ETA: 0s - loss: 0.1852 - accuracy: 0.91 - 1s 2ms/step - loss: 0.1852 - accuracy: 0.9123 - val_loss: 0.5777 - val_accuracy: 0.8360
Epoch 34/50
235/235 [==============================] - ETA: 0s - loss: 0.1277 - accuracy: 0.96 - ETA: 0s - loss: 0.1572 - accuracy: 0.93 - ETA: 0s - loss: 0.1685 - accuracy: 0.92 - ETA: 0s - loss: 0.1690 - accuracy: 0.92 - ETA: 0s - loss: 0.1698 - accuracy: 0.92 - ETA: 0s - loss: 0.1694 - accuracy: 0.92 - ETA: 0s - loss: 0.1783 - accuracy: 0.91 - ETA: 0s - loss: 0.1801 - accuracy: 0.91 - ETA: 0s - loss: 0.1799 - accuracy: 0.91 - 1s 2ms/step - loss: 0.1810 - accuracy: 0.9181 - val_loss: 

235/235 [==============================] - ETA: 0s - loss: 0.0820 - accuracy: 0.93 - ETA: 0s - loss: 0.1226 - accuracy: 0.94 - ETA: 0s - loss: 0.1177 - accuracy: 0.95 - ETA: 0s - loss: 0.1140 - accuracy: 0.95 - ETA: 0s - loss: 0.1150 - accuracy: 0.95 - ETA: 0s - loss: 0.1196 - accuracy: 0.94 - ETA: 0s - loss: 0.1206 - accuracy: 0.94 - ETA: 0s - loss: 0.1224 - accuracy: 0.94 - 0s 2ms/step - loss: 0.1225 - accuracy: 0.9456 - val_loss: 0.9188 - val_accuracy: 0.8200
Epoch 50/50
235/235 [==============================] - ETA: 0s - loss: 0.0984 - accuracy: 0.96 - ETA: 0s - loss: 0.1241 - accuracy: 0.95 - ETA: 0s - loss: 0.1054 - accuracy: 0.96 - ETA: 0s - loss: 0.1034 - accuracy: 0.95 - ETA: 0s - loss: 0.1052 - accuracy: 0.95 - ETA: 0s - loss: 0.1073 - accuracy: 0.95 - ETA: 0s - loss: 0.1088 - accuracy: 0.95 - ETA: 0s - loss: 0.1069 - accuracy: 0.95 - 0s 2ms/step - loss: 0.1079 - accuracy: 0.9512 - val_loss: 0.9353 - val_accuracy: 0.8140
Epoch 1/50
235/235 [==============================] - 

235/235 [==============================] - ETA: 0s - loss: 0.1697 - accuracy: 0.93 - ETA: 0s - loss: 0.2812 - accuracy: 0.88 - ETA: 0s - loss: 0.2832 - accuracy: 0.88 - ETA: 0s - loss: 0.2758 - accuracy: 0.88 - ETA: 0s - loss: 0.2732 - accuracy: 0.88 - ETA: 0s - loss: 0.2698 - accuracy: 0.88 - ETA: 0s - loss: 0.2764 - accuracy: 0.88 - ETA: 0s - loss: 0.2821 - accuracy: 0.88 - 0s 2ms/step - loss: 0.2878 - accuracy: 0.8808 - val_loss: 0.3667 - val_accuracy: 0.8504
Epoch 16/50
235/235 [==============================] - ETA: 0s - loss: 0.3029 - accuracy: 0.87 - ETA: 0s - loss: 0.2761 - accuracy: 0.89 - ETA: 0s - loss: 0.2823 - accuracy: 0.89 - ETA: 0s - loss: 0.2842 - accuracy: 0.88 - ETA: 0s - loss: 0.2850 - accuracy: 0.88 - ETA: 0s - loss: 0.2891 - accuracy: 0.87 - ETA: 0s - loss: 0.2844 - accuracy: 0.88 - ETA: 0s - loss: 0.2794 - accuracy: 0.88 - ETA: 0s - loss: 0.2821 - accuracy: 0.88 - ETA: 0s - loss: 0.2804 - accuracy: 0.88 - 1s 3ms/step - loss: 0.2816 - accuracy: 0.8840 - val_loss: 

...

Epoch 7/50
235/235 [==============================] - ETA: 0s - loss: 0.4888 - accuracy: 0.75 - ETA: 0s - loss: 0.3520 - accuracy: 0.86 - ETA: 0s - loss: 0.3549 - accuracy: 0.85 - ETA: 0s - loss: 0.3494 - accuracy: 0.86 - ETA: 0s - loss: 0.3384 - accuracy: 0.86 - ETA: 0s - loss: 0.3370 - accuracy: 0.86 - ETA: 0s - loss: 0.3366 - accuracy: 0.86 - ETA: 0s - loss: 0.3406 - accuracy: 0.86 - ETA: 0s - loss: 0.3417 - accuracy: 0.86 - ETA: 0s - loss: 0.3452 - accuracy: 0.85 - ETA: 0s - loss: 0.3465 - accuracy: 0.85 - ETA: 0s - loss: 0.3469 - accuracy: 0.85 - 1s 3ms/step - loss: 0.3450 - accuracy: 0.8587 - val_loss: 0.3654 - val_accuracy: 0.8536
Epoch 8/50
235/235 [==============================] - ETA: 0s - loss: 0.2073 - accuracy: 0.90 - ETA: 0s - loss: 0.3525 - accuracy: 0.85 - ETA: 0s - loss: 0.3470 - accuracy: 0.86 - ETA: 0s - loss: 0.3384 - accuracy: 0.86 - ETA: 0s - loss: 0.3379 - accuracy: 0.86 - ETA: 0s - loss: 0.3300 - accuracy: 0.86 - ETA: 0s - loss: 0.3287 - accuracy: 0.86 - ETA: 0

235/235 [==============================] - ETA: 0s - loss: 0.1803 - accuracy: 1.00 - ETA: 0s - loss: 0.2990 - accuracy: 0.87 - ETA: 0s - loss: 0.3092 - accuracy: 0.87 - ETA: 0s - loss: 0.3292 - accuracy: 0.86 - ETA: 0s - loss: 0.3237 - accuracy: 0.86 - ETA: 0s - loss: 0.3252 - accuracy: 0.86 - ETA: 0s - loss: 0.3242 - accuracy: 0.86 - ETA: 0s - loss: 0.3324 - accuracy: 0.87 - ETA: 0s - loss: 0.3273 - accuracy: 0.87 - ETA: 0s - loss: 0.3257 - accuracy: 0.87 - ETA: 0s - loss: 0.3257 - accuracy: 0.87 - ETA: 0s - loss: 0.3258 - accuracy: 0.87 - ETA: 0s - loss: 0.3270 - accuracy: 0.87 - ETA: 0s - loss: 0.3293 - accuracy: 0.86 - 1s 3ms/step - loss: 0.3293 - accuracy: 0.8692 - val_loss: 0.3595 - val_accuracy: 0.8524
Epoch 20/50
235/235 [==============================] - ETA: 0s - loss: 0.3582 - accuracy: 0.87 - ETA: 0s - loss: 0.3069 - accuracy: 0.87 - ETA: 0s - loss: 0.3179 - accuracy: 0.86 - ETA: 0s - loss: 0.3030 - accuracy: 0.87 - ETA: 0s - loss: 0.2996 - accuracy: 0.87 - ETA: 0s - loss: 

235/235 [==============================] - ETA: 0s - loss: 0.2865 - accuracy: 0.90 - ETA: 0s - loss: 0.2698 - accuracy: 0.88 - ETA: 0s - loss: 0.2611 - accuracy: 0.88 - ETA: 0s - loss: 0.2604 - accuracy: 0.88 - ETA: 0s - loss: 0.2658 - accuracy: 0.87 - ETA: 0s - loss: 0.2624 - accuracy: 0.88 - ETA: 0s - loss: 0.2627 - accuracy: 0.88 - ETA: 0s - loss: 0.2742 - accuracy: 0.87 - ETA: 0s - loss: 0.2768 - accuracy: 0.87 - ETA: 0s - loss: 0.2771 - accuracy: 0.87 - ETA: 0s - loss: 0.2780 - accuracy: 0.87 - ETA: 0s - loss: 0.2778 - accuracy: 0.87 - ETA: 0s - loss: 0.2799 - accuracy: 0.87 - 1s 3ms/step - loss: 0.2793 - accuracy: 0.8760 - val_loss: 0.3913 - val_accuracy: 0.8572
Epoch 32/50
235/235 [==============================] - ETA: 0s - loss: 0.2484 - accuracy: 0.93 - ETA: 0s - loss: 0.2392 - accuracy: 0.90 - ETA: 0s - loss: 0.2358 - accuracy: 0.90 - ETA: 0s - loss: 0.2516 - accuracy: 0.89 - ETA: 0s - loss: 0.2667 - accuracy: 0.88 - ETA: 0s - loss: 0.2730 - accuracy: 0.87 - ETA: 0s - loss: 

235/235 [==============================] - ETA: 0s - loss: 0.4270 - accuracy: 0.90 - ETA: 0s - loss: 0.2685 - accuracy: 0.89 - ETA: 0s - loss: 0.2436 - accuracy: 0.89 - ETA: 0s - loss: 0.2404 - accuracy: 0.89 - ETA: 0s - loss: 0.2395 - accuracy: 0.89 - ETA: 0s - loss: 0.2403 - accuracy: 0.89 - ETA: 0s - loss: 0.2447 - accuracy: 0.88 - ETA: 0s - loss: 0.2432 - accuracy: 0.88 - ETA: 0s - loss: 0.2444 - accuracy: 0.88 - ETA: 0s - loss: 0.2430 - accuracy: 0.88 - ETA: 0s - loss: 0.2444 - accuracy: 0.88 - ETA: 0s - loss: 0.2453 - accuracy: 0.88 - 1s 3ms/step - loss: 0.2457 - accuracy: 0.8879 - val_loss: 0.4351 - val_accuracy: 0.8340
Epoch 44/50
235/235 [==============================] - ETA: 0s - loss: 0.0663 - accuracy: 1.00 - ETA: 0s - loss: 0.2315 - accuracy: 0.90 - ETA: 0s - loss: 0.2323 - accuracy: 0.89 - ETA: 0s - loss: 0.2254 - accuracy: 0.89 - ETA: 0s - loss: 0.2343 - accuracy: 0.89 - ETA: 0s - loss: 0.2330 - accuracy: 0.89 - ETA: 0s - loss: 0.2399 - accuracy: 0.88 - ETA: 0s - loss: 

INFO:tensorflow:Oracle triggered exit
Wall time: 9min 58s


In [24]:
tuner.results_summary()